In [3]:
from __future__ import division
%pylab inline
import pandas as pd
import os
import json
from collections import defaultdict, Counter
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


# DATA
1. Get host IP and ASN for Alexa top 500 websites: use socket and pyasn (offline routeviews) libraries
- Find CDN of each website: use list of common CDNs and their related domains
    - Compare site to well known CDN domains (such as googleusercontent.com, cloudfront.net, etc.)
    - Parse homepage 
        - Parsing static objects on homepage to find number of resources per well known CDN domain
    - Parse whois
        - 'Organization' field might have the CDN name
        - site contact email servers used might be related to certain CDN domains
- Use curl to get timings for accessing sites: save response time for at least 20 requests per site
    - -v and --trace-time flags allow us to read timing information in real time
    - curl -w flag allows us to write time since the request was issued for name lookup, connection, SSL negotiation, and data reception
    - Calculate times:
        - t_dns = time for DNS resolution (no redirects) = time_namelookup - time_redirect  
        - t_tcp = time for TCP connection (SYN/SYNACK) = time_connect - time_namelookup  
        - t_ssl = time for SSL handshake (only if https) = time_appconnect - time_connect  
        - t_fbyte = time_starttransfer
        - t_wait = time between issuing GET request and first byte received = time_starttransfer - time_pretransfer  
        - t_rx = time to receive data from first to last byte = time_total - time_starttransfer

# Analysis
- 

In [4]:
df_sites = pd.read_csv('data/top-1m-new.csv', nrows=500, header = None, names = ['rank', 'site'])

## 1. Get IP and ASNs
- Get IP using socket.gethostbyname method
- Get ASN using pyasn (can also be done using whois, downloading routeviews data, etc.)

In [ ]:
import pyasn

asndb = pyasn.pyasn('data/ipasn_20181212.dat')  #downloaded pyasn_util_download.py --latest
print(asndb.lookup('8.8.8.8'))

def findASN(ip):
    if ip:
        return asndb.lookup(ip)[0]
    else:
        return False

In [ ]:
df_sites['ASN'] = df_sites['IP'].apply(findASN)

In [ ]:
df_sites.head()

In [27]:
from fetch_site_info import get_ip, get_asn
get_asn('216.58.221.46')

('15169', 'GOOGLE - Google LLC, US')

In [ ]:
%run fetch_site_info.py

In [28]:
df_data = pd.read_json("output/df_data.json")

In [31]:
from CDNdomains import cdn_names, cdn_domains
all_cdn_names = list(set(cdn_names + list(cdn_domains.values())))

In [33]:
#def find_cdn():
#    return

df_data

,rank,site,ip,asn,asname
0,1,google.com,216.58.221.46,15169,"GOOGLE - Google LLC, US"
1,2,youtube.com,172.217.167.46,15169,"GOOGLE - Google LLC, US"
2,3,facebook.com,157.240.25.35,32934,"FACEBOOK - Facebook, Inc., US"
3,4,baidu.com,220.181.57.216,23724,"CHINANET-IDC-BJ-AP IDC, China Telecommunicatio..."
4,5,wikipedia.org,103.102.166.224,14907,"WIKIMEDIA - Wikimedia Foundation Inc., US"
5,6,qq.com,111.161.64.48,4837,CHINA169-BACKBONE CHINA UNICOM China169 Backbo...
6,7,yahoo.com,98.137.246.7,36647,"YAHOO-GQ1 - Yahoo, US"
7,8,amazon.com,176.32.98.166,16509,"AMAZON-02 - Amazon.com, Inc., US"
8,9,taobao.com,140.205.220.96,37963,CNNIC-ALIBABA-CN-NET-AP Hangzhou Alibaba Adver...
9,10,reddit.com,151.101.129.140,54113,"FASTLY - Fastly, US"


### Separate unblocked IPs and sites for analysis
- 20 of 500 sites are unreachable or blocked by the ISP
- Querying these sites and urls returns an IP at the edge of the ISP we're connected to
- Most of these sites are porn related or known for adware/malware

In [ ]:
df_valid = df_sites[df_sites['IP'] != False]

print("Number of valid sites for further analysis: %s\n" % (len(df_valid)))

print("List of blocked sites: %s" % list(df_sites[df_sites['IP'] == False]['site']))

In [ ]:
# save site_to_IP dictionary for valid sites
df_sites.to_pickle('output/df_sites.pkl')

site_to_IP = (df_valid.set_index('site')['IP']).to_dict()
json.dump(site_to_IP, open('output/site_to_IP.json', 'w'))

## 2. Find CDN
- load data

In [2]:
df_sites = pd.read_pickle('output/df_sites.pkl')
df_valid = df_sites[df_sites['IP'] != False].copy()

### Estimate CDN
- CDNnames
- CDNdomains
- compare site to known CDN domains
- compare most popular url that hosts objects on site homepage to known CDN domains
- compare organization name from whois to CDN names
- compare email address from whois to CDN domains

In [3]:
from CDNdomains import cdn_domains, cdn_names
from find_cdn_methods import find_cdn_by_site, find_cdn_by_url, find_cdn_by_counting_url, find_cdn_by_whois_org

#copy CDNnames to CDNdomains.py
#with open('CDNnames.csv', 'r') as f:
#    CDNnames = f.read().splitlines()
    
site_to_IP = json.load(open('output/site_to_IP.json', 'r'))

In [4]:
#import logging
#logging.basicConfig(filename='output/log_cdn_finder.log', level=logging.DEBUG, 
#                    format='%(asctime)s - %(levelname)s - %(message)s')
#logging.debug('This is a log message.')

In [5]:
# split find cdn into 2 functions
def cdn_parse_site(site):
    """wrapper func parsing site name and urls on home page"""
    # site in cdn or cdn in site urls
    matching_cdn = find_cdn_by_site(site) or find_cdn_by_url(site) or find_cdn_by_counting_url(site)
    
    if not matching_cdn:  # False or "" are both returned as False
        return False

    return matching_cdn

def cdn_parse_whois(site):
    """wrapper func calling whois"""
    # whois org is cdn
    matching_cdn = find_cdn_by_whois_org(site)
    
    if not matching_cdn:  # False or "" are both returned as False
        #matching_cdn = find_cdn_by_whois_site(site)
        return False

    return matching_cdn

## Get CDN from both methods

In [6]:
df_valid['cdn_parsed'] = df_valid['site'].apply(cdn_parse_site)
df_valid['cdn_whois'] = df_valid['site'].apply(cdn_parse_whois)
df_valid['cdn'] = False

	baidu.com:  | 15: [('', 15)]
	wikipedia.org:  | 307: [('', 306), ('Google', 1)]
	qq.com:  | 67: [('', 67)]
	amazon.com: Amazon AWS | 104: [('Amazon AWS', 71), ('', 33)]
	reddit.com: www.redditstatic.com | 98: [('www.redditstatic.com', 69), ('', 29)]
	tmall.com: Alibaba | 71: [('', 59), ('Alibaba', 12)]
	twitter.com:  | 92: [('', 81), ('Twitter', 11)]
	live.com:  | 6: [('', 4), ('Microsoft', 1), ('Windows Azure', 1)]
	sohu.com: Sohu | 485: [('', 456), ('Sohu', 29)]
	jd.com:  | 135: [('', 135)]
	yandex.ru:  | 135: [('', 135)]
	weibo.com:  | 31: [('', 31)]
	sina.com.cn:  | 1806: [('', 1806)]
	360.cn:  | 273: [('', 272), ('qihucdn.com', 1)]
	login.tmall.com: Alibaba | 12: [('Alibaba', 9), ('', 3)]
	blogspot.com: Google | 83: [('Google', 42), ('', 41)]
	linkedin.com: LinkedIn CDN | 177: [('', 160), ('LinkedIn CDN', 17)]
	netflix.com: Open-Connect (Netflix) | 41: [('', 28), ('Open-Connect (Netflix)', 13)]
	yahoo.co.jp: Yahoo | 113: [('', 62), ('Yahoo', 51)]
	office.com: www.microsoft.com | 

	vice.com: vice-web-statics-cdn.vice.com | 157: [('', 88), ('vice-web-statics-cdn.vice.com', 63), ('Facebook', 2), ('Google', 2), ('cdn.roiq-vice.ranker.com', 1), ('vice-sundry-assets-cdn.vice.com', 1)]
	otvfoco.com.br:  | 314: [('', 303), ('Google', 7), ('Facebook', 2), ('cdn.onesignal.com', 1), ('Amazon CloudFront', 1)]
	bodelen.com: Google | 36: [('Google', 25), ('', 11)]
	fc2.com:  | 115: [('', 115)]
	uol.com.br:  | 207: [('', 206), ('Google', 1)]
	globo.com: s2.glbimg.com | 664: [('', 397), ('s2.glbimg.com', 266), ('tiqcdn.com', 1)]
	flipkart.com: rukminim1.flixcart.com | 67: [('', 47), ('rukminim1.flixcart.com', 17), ('Google', 2), ('Facebook', 1)]
	wetransfer.com: prod-cdn.wetransfer.net | 27: [('', 15), ('prod-cdn.wetransfer.net', 12)]
	mercadolivre.com.br:  | 208: [('', 201), ('Google', 6), ('Facebook', 1)]
	godaddy.com:  | 262: [('', 259), ('Google', 2), ('Facebook', 1)]
	quizlet.com:  | 65: [('', 63), ('Google', 1), ('Facebook', 1)]
	sciencedirect.com: www.elsevier.com | 5: 

	buzzfeed.com:  | 120: [('', 119), ('Google', 1)]
	gosuslugi.ru: gu-st.ru | 52: [('', 41), ('gu-st.ru', 11)]
	primevideo.com: Amazon AWS | 51: [('Amazon AWS', 34), ('', 17)]
	kinopoisk.ru:  | 12: [('', 12)]
	doublepimp.com: aspnetcdn.com | 9: [('', 5), ('aspnetcdn.com', 3), ('Google', 1)]
	rt.com: cdni.rt.com | 74: [('', 52), ('cdni.rt.com', 12), ('Google', 6), ('Facebook', 4)]
	ifeng.com:  | 1427: [('', 1427)]
	orange.fr: hp5.a.woopic.com | 187: [('', 125), ('hp5.a.woopic.com', 61), ('tiqcdn.com', 1)]
	redd.it: www.redditstatic.com | 140: [('www.redditstatic.com', 118), ('', 21), ('Google', 1)]
	zoho.com:  | 44: [('', 44)]
	fandom.com: fandom.wikia.com | 366: [('fandom.wikia.com', 302), ('Fastly', 22), ('', 21), ('Facebook', 13), ('Google', 8)]
	americanexpress.com:  | 346: [('', 346)]
	namnak.com:  | 97: [('', 95), ('Amazon CloudFront', 1), ('Google', 1)]
	yy.com: yyweb.yystatic.com | 58: [('', 23), ('yyweb.yystatic.com', 18), ('bs2cdn.yy.com', 17)]
	hp.com:  | 277: [('', 275), ('Goo

	dkn.tv:  | 593: [('', 585), ('Google', 5), ('Facebook', 3)]
	wix.com:  | 132: [('', 124), ('Google', 5), ('Facebook', 2), ('cdn.trackjs.com', 1)]
	siteadvisor.com:  | 8: [('', 8)]
	zippyshare.com:  | 52: [('', 52)]
	icloud.com:  | 16: [('', 16)]
	ndtv.com: gadgets360cdn.com | 676: [('', 646), ('gadgets360cdn.com', 22), ('cdn.ndtv.com', 5), ('Facebook', 2), ('cdn.gadgets360.com', 1)]
	chinadaily.com.cn:  | 475: [('', 474), ('Facebook', 1)]
	factaholics.com:  | 123: [('', 120), ('Google', 1), ('Facebook', 1), ('jsDelivr', 1)]
	olx.ua:  | 6: [('', 6)]
	wiktionary.org:  | 160: [('', 160)]
	livedoor.com:  | 112: [('', 112)]
	pikabu.ru:  | 20: [('', 20)]
	3dmgame.com:  | 785: [('', 785)]
	uidai.gov.in:  | 126: [('', 107), ('Google', 16), ('Facebook', 3)]
	airbnb.com:  | 86: [('', 86)]
	icicibank.com:  | 88: [('', 87), ('Facebook', 1)]
	patreon.com:  | 59: [('', 53), ('Cloudflare', 2), ('Facebook', 2), ('cdn.ravenjs.com', 1), ('Google', 1)]
	shaparak.ir:  | 41: [('', 41)]
	upwork.com:  | 41:

	whois ebay-kleinanzeigen.de | ['Marktplaats', 'ORG-MB40-RIPE', 'Marktplaats B.V.'] | 
	whois doubleclick.net | ['GOOGLE', 'Google LLC (GOGL)', 'Google LLC'] |  cdn Google
	whois reverso.net | ['SOFTISSIMO'] | 
	whois mega.nz | ['MEGA_LIMITED'] | 
	whois ltn.com.tw | ['NCICNET-NET'] | 
	whois yelp.com | ['NON-RIPE-NCC-MANAGED-ADDRESS-BLOCK', 'SKYCA-3', 'Fastly (SKYCA-3)', 'Fastly'] |  cdn Fastly
	whois rednet.cn | ['TencentCloud'] | 
	whois ci123.com | ['CHINANET-JS'] | 
	whois ups.com | ['ERX-NETBLOCK', 'UPS', 'UNITED PARCEL SERVICE (UPS-9)', 'UNITED PARCEL SERVICE'] | 
	whois bestbuy.com | ['AKAMAI', 'Akamai Technologies, Inc. (AKAMAI)', 'Akamai Technologies, Inc.'] |  cdn Akamai
	whois genius.com | ['CLOUDFLARENET', 'Cloudflare, Inc. (CLOUD14)', 'Cloudflare, Inc.'] |  cdn Cloudflare
	whois udemy.com | ['CLOUDFLARENET', 'Cloudflare, Inc. (CLOUD14)', 'Cloudflare, Inc.'] |  cdn Cloudflare
	whois washingtonpost.com | ['AKAMAI', 'Akamai Technologies, Inc. (AKAMAI)', 'Akamai Technologies,

In [7]:
all_valid_cdn_names = list( cdn_domains.values() ) + cdn_names
site_cdn_method_map = defaultdict(list)
i = 0

def estimate_cdn(site, cdn_parsed, cdn_whois):
    """logic to decide which CDN finally"""

    #if cdn_whois in ['Fastly', 'Cloudflare', 'CloudFront', 'Akamai', 'Alibaba', 'Google']:
    #    matching_cdn = cdn_whois
    
    if cdn_parsed and cdn_whois:  # confusion case when both exist
        if (cdn_parsed in all_valid_cdn_names):
            # known CDN not an unknown url
            matching_cdn = cdn_parsed
        else:
            matching_cdn = cdn_whois
            # note association between cdn_parsed of url type and cdn_whois
            global site_cdn_method_map
            site_cdn_method_map['site'].append(site)
            site_cdn_method_map['cdn_parsed'].append(cdn_parsed)
            site_cdn_method_map['cdn_whois'].append(cdn_whois)
            
    else:
        # assume cdn_whois has higher priority if cdn_parsed is an unknown cdn url - most will be amazon aws
        matching_cdn = cdn_whois or cdn_parsed
        
    """
    global i
    if cdn_parsed and cdn_whois:
        if cdn_parsed != cdn_whois:
            i += 1
            print(i, ". ", site, ":", matching_cdn, "<", cdn_parsed, "|", cdn_whois, ">")
    """
    
    return matching_cdn

In [8]:
df_valid['cdn'] = df_valid.apply(lambda row: estimate_cdn(row['site'], row['cdn_parsed'], row['cdn_whois']), axis=1)

In [9]:
df_site_cdn_method_map = pd.DataFrame(site_cdn_method_map)
df_site_cdn_method_map.to_pickle('output/df_site_cdn_method_map.pkl')

df_site_cdn_method_map

,site,cdn_parsed,cdn_whois
0,reddit.com,www.redditstatic.com,Fastly
1,office.com,www.microsoft.com,Microsoft
2,twitch.tv,twitchcdn.net,Fastly
3,t.co,twitter.com,Twitter
4,stackoverflow.com,cdn.sstatic.net,Fastly
5,porn555.com,porn555.agat-tech.com,Cloudflare
6,espn.com,espncdn.com,Amazon Technologies Inc.
7,quora.com,quoracdn.net,Amazon Technologies Inc.
8,amazonaws.com,aws.amazon.com,"Amazon.com, Inc."
9,bbc.com,www.bbc.co.uk,Fastly


In [10]:
df_valid.to_pickle('output/df_asn_cdn.pkl')

# NOTES

- Contendo may be part of Akamai but we haven't replaced it
- instructure.com probably not on CDN, but whois detects Amazon
- nih.gov is on Amazon AWS even though all links are in fact local AS16509
- nextoptim.com detected as Google since we are redirected to google's home page but is actually a redirect virus
- homedepot.com 35.201.95.83 whois shows google cloud customer but might be Akamai's old customer (like apple?)
- airbnb.com should be using Fastly, but whois shows only Amazon, so detected as Amazon
- bitly.com uses amazon cloudfront probably but whois doesn't show. should have been detected in site url count.
- alipay.com uses alicdn.com but we are unable to grab urls as they are deep inside css style div backgroundimage strings
- office365.com is auth page not on CDN, but once logged in all data will be served by akamai. should this be included? added to CDN_domains.py
- twitch.tv will give twitchcdn.net if page parse first and fastly if whois first. twitchcdn.net belongs to fastly
- office.com will give Microfoft on whois and Akamai on pageparse
- cdn whois Amazon Technologies Inc. is infact Amazon EC2 instance of wepage. Amazon EC2 may not use Cloudfront to host content files, though its recomended if you want lot of content delivery
- spotify has moved to google cloud
- it seems tokopedia.com was Cloudfront but has moved to Alibaba Cloud (as seen on whois)

- <font color='red'>For analysis "Amazon Technologies Inc.", "Amazon AWS" and "Amazon Cloudfront", "Cloudfront" should all resolve to "Cloudfront"
- tripadvisor.com whois shows MCI Communications Services, Inc. d/b/a Verizon Business (MCICS) and resolves to Verizon as CDN name. In fact it is an entry for verizon EdgeCast. In future all Verizon should be resolved to "EdgeCase"
</font>

## Case for parsing page
- whois force.com  shows 'Internet Media Network', 'UltraDNS Corp'. UltraDNS is a DNS service probably not cdn. However it should be on Salesforce/Akamai as detected by parser
- whois researchgate.net and roblox.com show 'Cogeco Peer 1'. This CDN has not been detected by parser. Parser shows that its not a CDN site.
- whois popads.net shows Total Uptime Technologies. its cloud balancing service not CDN. popads.net should not be on cdn.
- <font color='red'>whois txxx.com and hclips.com gives cloudflare but page parsing pix-cdn.net is linked to something called 'Advanced Hosters CDN'. This is only copied from a previous file and is unconfirmed. Should we remove it???</font>

## TODO


# NOTES2: Blocked, redirects, and unreachable
- Blocked lookups at ISP
    - pornhub.com, livejasmin.com, xvideos.com, xhamster.com, xnxx.com, chaturbate.com, youporn.com, redtube.com, sex.com, 
    - yts.am : movie torrents
    - ok.ru
    - 1337x.to : movie torrents
    - rutracker.org : torrents
- Blocked malware (403 Forbidden)
    - exosrv.com : malware/adware
    - swiftviz.net : virus redirect/malware
    - i62e2b4mfy.com : malware/adware
    - vinfdv6b4j.com : virus redirect
    - s9kkremkr0.com : virus redirect
    - omumultation.club : virus popups
    - nextoptim.com : adware redirect popups
    - resentaticexhaus.info : possible malware, IP timeouts on ping, not 403
    - exdynsrv.com : adware redirect/malware, not 403
    - hotchedmothe.club: malware popup ads
    - digitaldsp.com : redirect virus, not 403
    - jf71qh5v14.com : virus/adware
    -
- No lookup or redirect
    - banvenez.com > e-bdvcpx.banvenez.com exists but can't be reached automatically
    
    
- Unreachable servers (possible CDNs)
    - microsoftonline.com : port 80 and 443 are probably closed
    - googleusercontent.com :  port 80 and 443 are probably closed
    - twimg.com : Twitter CDN by Twitter (twimg.com) blocked on port 80 and 443
    - cloudfront.net > hosted on Amazon aws as part of Amazon Cloudfront CDN. Blocked port 80 and 443. No DNS lookup.
    - bp.blogspot.com > hosts resources for blogspot. Blocked 80 443. Probably not CDN just a simple direct server.
        - https://2-bp.blogspot.com/2017/11/what-are-bpblogspotcom-links-and-how.html 
        
    
- Redirects
    - wikia.com > wikia.com/fandom
    - zhihu.com > zhihu.com/signup
    - fandom.com > fandom.wikia.com
        - contains fastly-insights.com widget scripts
    - rednet.cn > can't really reach this page on browser either
        - downloads a 200 response page saying redirect
     - ci123.com > same as above
         - 200 response redirect page that is very slow on browser was manually downloaded to ci123.com.html
- No curl (timeouts with curl not with requests)
    - usps.com
    - bestbuy.com
    - udemy.com > robot check
    - momoshop.com.tw > different source when viewed online vs by curl or requests
        - 1 small asia.creativecdn.com in script tag on end of page, most sources on momoshop.com.tw
        - manually download main page from browser as momoshop.com.tw.html
    - leboncoin.fr > blocked 403 when using requests/curl but available on browser
        - manually download main page from browser as leboncoin.fr.html
    - kissanime.ru > Please wait 5 seconds... doesn't work with minibrowsers
        - manually download main page from browser as kissanime.ru.html
    - roblox.com > gaming site that prechecks for cookies and scripts and tells not working with 200 code
        - manually download main page from browser as roblox.com.html
- Others
    - jianshu.com > cdn2.jianshu.com in < link href >
    - shopify.com > cdn.shopify.com in < link href >
    - xfinity.com > cdn.comcast.com
    - investing.com  > link href akamaized.net
    - discordapp.cdn > has a field CDN_HOST: 'cdn.discordapp.com' but not in static sources
    - bodelen.com > undetected redirect adware virus shows google page
    - netflix.com > video content is on Open-Connect CDN however homepage doesn't load any video content

In [ ]:
#site_to_IP = df_valid.set_index('site')['IP'].to_dict()

# after parsing whois info from CDN_BGP_Analysis
#sites_org = pd.read_pickle('output/df_org_email_info.pkl')

#sites_org = df_valid['IP'].apply(lambda s: pd.Series( findOrg(s) ) )
#sites_cdn = df_valid.merge(sites_org, left_index=True, right_index=True )

# 3. Get curl request timings

In [46]:
df_asn_cdn = pd.read_pickle('output/df_asn_cdn.pkl')

In [102]:
df_asn_cdn.head()

,rank,site,IP,ASN,cdn_parsed,cdn_whois,cdn
0,1,google.com,172.217.160.238,15169,Google,Google,Google
1,2,youtube.com,172.217.31.14,15169,Google,Google,Google
2,3,facebook.com,157.240.25.35,32934,Facebook,Facebook,Facebook
3,4,baidu.com,123.125.115.110,4808,False,False,False
4,5,wikipedia.org,103.102.166.224,14907,False,False,False


In [43]:
dfile = "output/site_to_ip.json"
os.path.exists(dfile)
with open(dfile, 'r') as fin:
    site_to_ip = json.load(fin)

In [44]:
site_to_ip

{'google.com': '172.217.160.238',
 'youtube.com': '172.217.31.14',
 'facebook.com': '157.240.25.35',
 'baidu.com': '123.125.115.110',
 'wikipedia.org': '103.102.166.224',
 'qq.com': '111.161.64.48',
 'yahoo.com': '72.30.35.10',
 'amazon.com': '176.32.98.166',
 'taobao.com': '140.205.94.189',
 'reddit.com': '151.101.129.140',
 'tmall.com': '140.205.130.99',
 'google.co.in': '172.217.166.195',
 'twitter.com': '104.244.42.65',
 'live.com': '204.79.197.212',
 'sohu.com': '221.179.177.36',
 'jd.com': '120.52.148.118',
 'yandex.ru': '5.255.255.55',
 'google.co.jp': '172.217.31.3',
 'instagram.com': '52.45.71.129',
 'weibo.com': '114.134.80.162',
 'sina.com.cn': '202.108.33.107',
 '360.cn': '36.110.213.49',
 'login.tmall.com': '223.30.211.253',
 'blogspot.com': '172.217.161.9',
 'google.com.hk': '172.217.160.227',
 'linkedin.com': '108.174.10.10',
 'netflix.com': '54.76.161.146',
 'google.com.br': '172.217.31.3',
 'yahoo.co.jp': '183.79.135.206',
 'office.com': '52.108.36.3',
 'microsoftonlin

In [46]:
site_to_IP = json.load(open('output/site_to_ip.json', 'r'))

## check cdn estimation method
- check out output of estimate_cdn.py
- many non google domains estimated as google due to googleadservices
- since we delete "none" => what are local links '' from netloc???
    - wikia.com:  | 383: [('', 340), ('Fastly', 22), ('Facebook', 13), ('Google', 8)]
    - bukalapak.com: Google | 437: [('', 426), ('Google', 6), ('Facebook', 4), ('cdn.onesignal.com', 1)]
    - onlinesbi.com: Google | 96: [('', 90), ('Google', 4), ('Facebook', 2)]
    - walmart.com: Google | 217: [('', 210), ('Google', 4), ('Facebook', 2), ('Twitter', 1)]
    - mercadolivre.com.br: Google | 208: [('', 201), ('Google', 6), ('Facebook', 1)]
- maybe change CDNdomains file to reflect google.\* vs ads??
- try getting script directly from cdn??
- return confidence value from netloc counter (numlinks/total)
- get rid of <a href> tag in the counter?

In [137]:
for rank, site in rank_to_site.items():
    print("\n", rank,site)
    a = get_homepage_url_count(site)


 1 google.com
.png||img src|/logos/doodles/2018/the-geminid-meteor-shower-2018-5112685963247616.3-l.png

 2 youtube.com
.js||script src|/yts/jsbin/scheduler-vfl2iJIO4/scheduler.js
.js||script src|/yts/jsbin/spf-vflhSOzLf/spf.js
.js||script src|/yts/jsbin/www-en_US-vflmmpDne/base.js
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.gif||img src|/yts/img/pixel-vfl3z5WfW.gif
.jpg|i.ytimg.com|img src|https://i.ytimg.com/vi/iP2mL74yVB4/hqdefault.jpg?sqp=-oaymwEiCMQBEG5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==&rs=AOn4CLCerThSj7fZ6vlGJiST0A3nffr1sQ
.jpg|i.ytimg.com|img src|https://i.ytimg.com/vi/_KhQT-LGb-4/hqdefault.jpg?sqp=-oaymwEiCMQBEG5IWvKriqkDFQgBFQAAAAA

.js||script src|portal/wikipedia.org/assets/js/index-aaace14cc2.js
.js||script src|portal/wikipedia.org/assets/js/gt-ie9-a2995951ca.js
.png||img src|portal/wikipedia.org/assets/img/Wikipedia-logo-v2.png
.png||link href|/static/apple-touch/wikipedia.png
.ico||link href|/static/favicon/wikipedia.ico
.svg||link href|portal/wikipedia.org/assets/img/sprite-556af1a5.svg

 6 qq.com
.png|mat1.gtimg.com|img src|//mat1.gtimg.com/pingjs/ext2020/qqindex2018/dist/img/qq_logo_2x.png
.png|img1.gtimg.com|img src|//img1.gtimg.com/ninja/2/2018/10/ninja153908078197426.png
.png|img1.gtimg.com|img src|//img1.gtimg.com/ninja/2/2018/10/ninja153908079663519.png
.png|img1.gtimg.com|img src|//img1.gtimg.com/ninja/2/2018/10/ninja153907290259802.png
.png|img1.gtimg.com|img src|//img1.gtimg.com/ninja/2/2018/10/ninja153907291410277.png
.ico|mat1.gtimg.com|link href|//mat1.gtimg.com/www/icon/favicon2.ico
.css|mat1.gtimg.com|link href|//mat1.gtimg.com/pingjs/ext2020/qqindex2018/dist/css/qq_90e23e2b.css

 7 yahoo.com


.js|statics.itc.cn|script src|//statics.itc.cn/web/v3/static/js/lib-61587d9fb8.js
.js|39d0825d09f05.cdn.sohucs.com|script src|//39d0825d09f05.cdn.sohucs.com/sdk/passport-4.0.3.js
.js|statics.itc.cn|script src|//statics.itc.cn/web/v3/static/js/main-5b663e7c70.js
.js|js.sohu.com|script src|//js.sohu.com/libpv.js
.js|images.sohu.com|script src|//images.sohu.com/bill/default/sohu-require.js
.js|www.sohu.com|script src|//www.sohu.com/sohuflash_1.js
.js|images.sohu.com|script src|//images.sohu.com/bill/s2015/jscript/lib/sjs/matrix/ad/form/delivery.js
.js|images.sohu.com|script src|//images.sohu.com/bill/s2015/jscript/lib/sjs/matrix/pv/pagePVmonitor.js
.jpg|29e5534ea20a8.cdn.sohucs.com|img src|http://29e5534ea20a8.cdn.sohucs.com/c_zoom,h_213/c_cut,x_4,y_2,w_494,h_330/os/news/39fb99870db69e253ea3c4e572210c21.jpg
.jpg|29e5534ea20a8.cdn.sohucs.com|img src|http://29e5534ea20a8.cdn.sohucs.com/c_zoom,h_213/c_cut,x_50,y_0,w_418,h_279/os/news/ff536d5c344701975f9ecec8eb5e8a9e.jpg
.jpg|29e5534ea20a8.cd

.js|d3.sina.com.cn|script src|//d3.sina.com.cn/litong/zhitou/sinaads/release/sinaads.js
.js|news.sina.com.cn|script src|//news.sina.com.cn/js/pctianyi/sima.js
.js|n.sinaimg.cn|script src|//n.sinaimg.cn/ent/js/lib/jquery-1.7.2.js
.js|d3.sina.com.cn|script src|//d3.sina.com.cn/litong/zhitou/sinaads/demo/wanglt/sinaIndexAD/edu_ad_change.js
.js|i.sso.sina.com.cn|script src|//i.sso.sina.com.cn/js/ssologin.js
.js|news.sina.com.cn|script src|//news.sina.com.cn/sso/outlogin_layer.js
.js|news.sina.com.cn|script src|//news.sina.com.cn/sso/user_panel_homepage.js
.js|auto.sina.com.cn|script src|//auto.sina.com.cn/sinaauto/2016/sinahome/chooseCars.js
.js|d3.sina.com.cn|script src|//d3.sina.com.cn/d1images/sinaads_entry/sinaads_entry_index.js
.js|www.sina.com.cn|script src|//www.sina.com.cn/js/79/2013/0717/fix.js
.js|d3.sina.com.cn|script src|//d3.sina.com.cn/litong/zhitou/sinaads/release/sinaads.js
.js|d3.sina.com.cn|script src|//d3.sina.com.cn/litong/zhitou/sinaads/demo/wanglt/sinaIndexAD/sinaAD_s

.js|g.alicdn.com|script src|//g.alicdn.com/secdev/pointman/js/index.js
.js|g.alicdn.com|script src|//g.alicdn.com//vip/login/0.5.36/js/jump.js
.png|img.alicdn.com|img src|//img.alicdn.com/tfs/TB1_Gn8RXXXXXXqaFXXXXXXXXXX-380-54.png
.ico|g.alicdn.com|link href|//g.alicdn.com/mui/global/1.2.35/file/favicon.ico
.xml|g.alicdn.com|link href|//g.alicdn.com/mui/global/1.2.35/file/search.xml
.css|g.alicdn.com|link href|//g.alicdn.com/mui/global/3.0.31/global.css

 24 blogspot.com
.js|ajax.googleapis.com|script src|//ajax.googleapis.com/ajax/libs/webfont/1.5.10/webfont.js
.js||script src|js/main.min.js?v=1476060773
.png||link href|favicon/apple-touch-icon-57x57.png
.png||link href|favicon/apple-touch-icon-60x60.png
.png||link href|favicon/apple-touch-icon-72x72.png
.png||link href|favicon/apple-touch-icon-76x76.png
.png||link href|favicon/apple-touch-icon-114x114.png
.png||link href|favicon/apple-touch-icon-120x120.png
.png||link href|favicon/apple-touch-icon-144x144.png
.png||link href|favicon/

...

.jpg|a.zdmimg.com|img src|https://a.zdmimg.com/201812/06/5c092cbe196173383.jpg_a200.jpg
.jpg|a.zdmimg.com|img src|https://a.zdmimg.com/201812/08/5c0b732a8d7ac3912.jpg_a200.jpg
.jpg|qna.smzdm.com|img src|https://qna.smzdm.com/201812/10/5c0dd70e40f7e4685.jpg_a200.jpg
.jpg|qna.smzdm.com|img src|https://qna.smzdm.com/201812/10/5c0e1347617615338.jpg_a200.jpg
.jpg|qna.smzdm.com|img src|https://qna.smzdm.com/201812/12/5c106bfa8d07c6324.jpg_a200.jpg
.jpg|qna.smzdm.com|img src|https://qna.smzdm.com/201812/11/5c0f81f73bc172220.jpg_a200.jpg
.jpg|qna.smzdm.com|img src|https://qna.smzdm.com/201812/11/5c0fcacadd0234581.jpg_a200.jpg
.jpg|a.zdmimg.com|img src|https://a.zdmimg.com/201812/11/5c0f1e09ae0ab8488.jpg_a200.jpg
.jpg|qnz.smzdm.com|img src|https://qnz.smzdm.com/201812/05/5c07b1e6ec6161525.png_a200.jpg
.jpg|z.zdmimg.com|img src|https://z.zdmimg.com/201812/07/5c0a51d592d6f8150.png_a200.jpg
.jpg|z.zdmimg.com|img src|https://z.zdmimg.com/201812/05/5c076b6f4515d5277.png_a200.jpg
.jpg|qnz.smzdm.com|i

.js|www.ebay.com|script src|//www.ebay.com/scl/js/ScandalLoader.js
.js|ir.ebaystatic.com|script src|https://ir.ebaystatic.com/rs/c/inception-dnRHPO-V.js
.js|ir.ebaystatic.com|script src|https://ir.ebaystatic.com/rs/c/highlnfe-ua7ccP4C.js
.js|ir.ebaystatic.com|script src|https://ir.ebaystatic.com/rs/v/10341xh50yz21mhhydueu4m5wad.js
.js|ir.ebaystatic.com|script src|https://ir.ebaystatic.com/rs/v/it02syay0qyozhdaszhv1jl4yyd.js
.js|ir.ebaystatic.com|script src|https://ir.ebaystatic.com/rs/c/dfp/v4.2.js
.js|ir.ebaystatic.com|script src|https://ir.ebaystatic.com/rs/v/vsmpn5ngce2efnq1ckwdvnumxqf.js
.js|ir.ebaystatic.com|script src|https://ir.ebaystatic.com/rs/c/makeebayfasterscript-src-scripts-body-78a2168a.js
.png|ir.ebaystatic.com|img src|https://ir.ebaystatic.com/cr/v/c1/red-logo.min.png
.svg|ir.ebaystatic.com|img src|https://ir.ebaystatic.com/cr/v/c1/eBayPlus/ebayPlus-Logo-58x15.svg
.gif|ir.ebaystatic.com|img src|//ir.ebaystatic.com/pictures/aw/pics/s_1x2.gif
.jpg|i.ebayimg.com|img src|ht

.png|s.blogsmithmedia.com|link href|https://s.blogsmithmedia.com/www.oath.com/assets-h440030358d79d708bb881e7c8107c8e7/images/favicons/oathfavicon_16x16_aa_v2.png?h=e0fe0b538e58e69fa819790fdc7cc639
.png|s.blogsmithmedia.com|link href|https://s.blogsmithmedia.com/www.oath.com/assets-h440030358d79d708bb881e7c8107c8e7/images/favicons/oathfavicon_32x32_aa_v2.png?h=d25852b6d1192559e10a591238552633
.png|s.blogsmithmedia.com|link href|https://s.blogsmithmedia.com/www.oath.com/assets-h440030358d79d708bb881e7c8107c8e7/images/favicons/oathfavicon_64x64_aa_v2.png?h=5eb18974b8261bfb6d44daa2409d4b82
.ico|s.blogsmithmedia.com|link href|https://s.blogsmithmedia.com/www.oath.com/assets-h440030358d79d708bb881e7c8107c8e7/images/favicons/Oath_Favicon.ico?h=9e9bd855b6dae80baa2af84d0d2bdccd
.css|s.blogsmithmedia.com|link href|https://s.blogsmithmedia.com/www.oath.com/assets-h440030358d79d708bb881e7c8107c8e7/libraries/font-awesome-4.7.0/css/font-awesome.min.css?h=269550530cc127b6aa5a35925a7de6ce
.css|s.blog

.gif|d5nxst8fruw4z.cloudfront.net|img src|https://d5nxst8fruw4z.cloudfront.net/atrk.gif?account=cFNEk1ao6C52uG
.png||img src|/assets/bbcl-escritorio-nuevo/img/BIOBIOCHILEBLANCO.png
.svg||img src|/assets/biobiochile/img/logos/BIOBIOCHILE.svg
.svg||img src|/assets/biobiochile/img/logos/Facebook_36_Color.svg
.svg||img src|/assets/biobiochile/img/logos/Twitter_36_Color.svg
.svg||img src|/assets/biobiochile/img/logos/Instagram_36_Color.svg
.svg||img src|/assets/biobiochile/img/logos/Youtube_36_Color.svg
.gif||img src|/assets/biobiochile/img/ajax-loader.gif
.png||img src|/assets/barra-g20-2018/barra_escritorio.png
.png||img src|/assets/bbcl-escritorio-nuevo/img/barra-DW-esc_730x50@2x.png
.png||img src|/assets/bbclnew/img/favicon-BBCL.png
.png|www.biobiochile.cl|img src|https://www.biobiochile.cl/assets/biobiochile/img/flag-UK-esc.png
.png||img src|/assets/biobiochile/img/bg-fena-gonzalez-02.png
.png|www.biobiochile.cl|img src|https://www.biobiochile.cl/assets/bbcl-escritorio-nuevo/img/iso-bb

.js||script src|/static/project/adp/js/userprivacysettings.js
.js|cdn.optimizely.com|script src|//cdn.optimizely.com/js/198303326.js
.js||script src|/-/media/adp/Redesign2018/modernizr/modernizr-custom.js
.js||script src|/js/s_code.js
.js||script src|/sitecore%20modules/Web/Web%20Forms%20for%20Marketers/mvc/libs/jquery/jquery-2.1.3.min.js
.js||script src|/sitecore%20modules/Web/Web%20Forms%20for%20Marketers/mvc/libs/jquery/jquery-ui-1.11.3.min.js
.js||script src|/sitecore%20modules/Web/Web%20Forms%20for%20Marketers/mvc/libs/jquery/jquery.validate.min.js
.js||script src|/sitecore%20modules/Web/Web%20Forms%20for%20Marketers/mvc/libs/jquery/jquery.validate.unobtrusive.min.js
.js||script src|/sitecore%20modules/Web/Web%20Forms%20for%20Marketers/mvc/libs/bootstrap/bootstrap.min.js
.js||script src|/sitecore%20modules/Web/Web%20Forms%20for%20Marketers/mvc/adpEarlyValidation.js
.js||script src|/sitecore%20modules/Web/Web%20Forms%20for%20Marketers/mvc/wffm.min.js
.js||script src|/sitecore%20mod

In [154]:
import urllib.parse
from bs4 import BeautifulSoup

def get_parsed_info(loc):
    """netloc is base url"""
    if loc is None or loc is "":  # empty src or link instead of local src or link
        return None, None
    parsed = urllib.parse.urlparse(loc)
    fname, fextension = os.path.splitext(parsed.path)
    
    return fextension, parsed.netloc

#ext = []

def count_netlocs(data):
    """find static resources on a webpage and urls they are served from"""
    soup = BeautifulSoup(data, features="lxml")

    #netlocs = defaultdict(list)
    netlocs_static = []
    tags_to_search = ["script src", "img src", "source srcset", "source data-srcset",
                      "link href",]    # "meta content", "a href", ]
    #extensions_to_count = [".js", ".css", ".jsp",
    #                       ".png", ".jpeg", ".jpg", ".gif", ".img",
    #                       ".ico",".woff",".woff2",".svg",]    # ".xml",".json", ".rss"
    #global ext
    for tag in tags_to_search:
        html_tag, html_src = tag.split(" ")
        html_elem = soup.findAll(html_tag)
        for elem in html_elem:
            obj = elem.get(html_src)
            obj_type, obj_loc = get_parsed_info(obj)
            if obj_type is not None and obj_type is not "":
                #netlocs[obj_type].append(obj_loc)
                #print(obj_type, obj_loc, tag, obj, sep="|")
                netlocs_static.append(obj_loc.strip())
                #ext.append(obj_type)
                #if obj_type.lower() in extensions_to_count:
                #    netlocs_static.append(obj_loc)
    
    """
    # all
    tags = soup.findAll('a')
    for tag in tags:
        loc = tag.get('href')
        if not (nloc == ''):
            netlocs_all.append( getnetloc(loc) )
    """
    return Counter(netlocs_static)

def get_homepage_url_count(site):
    """counts url static resources in homepage
    input: site
    output: {url:count} in order where url is the base url from resources on homepage"""
    file = 'output/homepage/' + site
    altfile = 'output/homepage/' + site + '.html'

    if os.path.exists(file):
        with open(file, 'r') as f:
            data = f.read()
    elif os.path.exists(altfile):
        with open(altfile, 'r', encoding="utf8", errors='ignore') as f:
            data = f.read()
    else:
        # bad_sites.append(site)
        return False
    cnt = count_netlocs(data)
    del cnt[None]  # remove empty src NOT local src
    
    return cnt.most_common()

In [156]:
c = [1,1,1,1,1,11,2,2,2,4,3,4,5,6,6,3,2,1,3,3,55,1,2,34,4,2,22,1,3,1,2,3,12,12,2333,4]

d = Counter(c).most_common()


In [143]:
rank_to_site = df_sites.set_index("rank")["site"].to_dict()

In [144]:
for rank, site in rank_to_site.items():
    c = get_homepage_url_count(site)
    print("\n", rank,site,c)


 1 google.com [('', 1)]

 2 youtube.com [('', 70), ('i.ytimg.com', 33), ('s.ytimg.com', 1)]

 3 facebook.com [('static.xx.fbcdn.net', 13), ('facebook.com', 1), ('', 1)]

 4 baidu.com [('www.baidu.com', 2), ('s1.bdstatic.com', 1)]

 5 wikipedia.org [('', 6)]

 6 qq.com [('img1.gtimg.com', 4), ('mat1.gtimg.com', 3)]

 7 yahoo.com [('s.yimg.com', 39), ('in.search.yahoo.com', 1)]

 8 amazon.com [('images-na.ssl-images-amazon.com', 69)]

 9 taobao.com [('img.alicdn.com', 3), ('g.alicdn.com', 2)]

 10 reddit.com [('www.redditstatic.com', 66)]

 11 tmall.com [('img.alicdn.com', 4), ('g.alicdn.com', 2)]

 12 google.co.in [('', 1)]

 13 twitter.com [('abs.twimg.com', 9), ('', 2)]

 14 live.com [('r1.res.office365.com', 1), ('az725175.vo.msecnd.net', 1)]

 15 sohu.com [('statics.itc.cn', 84), ('5b0988e595225.cdn.sohucs.com', 18), ('29e5534ea20a8.cdn.sohucs.com', 10), ('images.sohu.com', 3), ('www.sohu.com', 2), ('39d0825d09f05.cdn.sohucs.com', 1), ('js.sohu.com', 1)]

 16 jd.com [('wl.jd.com', 


 118 cnblogs.com [('pic.cnblogs.com', 16), ('common.cnblogs.com', 5), ('', 3), ('www.googletagservices.com', 1)]

 119 stackexchange.com [('cdn.sstatic.net', 48), ('i.stack.imgur.com', 7), ('', 5), ('ajax.googleapis.com', 1)]

 120 cnn.com [('cdn.cnn.com', 17), ('', 11), ('edition.i.cdn.cnn.com', 4), ('cdn.optimizely.com', 3), ('a.postrelease.com', 2), ('native.sharethrough.com', 1), ('static.chartbeat.com', 1), ('cdn.cookielaw.org', 1), ('pixel.quantserve.com', 1), ('amplify.outbrain.com', 1), ('com.cnn.mobile.android.phone', 1)]

 121 aparat.com [('www.aparat.com', 22), ('cdn.ravenjs.com', 1), ('cdn.onesignal.com', 1), ('logo.samandehi.ir', 1), ('certify.alexametrics.com', 1)]

 122 researchgate.net [('c5.rgstatic.net', 35), ('www.researchgate.net', 11), ('i1.rgstatic.net', 1)]

 123 detik.com [('awsimages.detik.net.id', 82), ('awscdn.detik.net.id', 67), ('awscdnstatic.detik.net.id', 4), ('static.criteo.net', 1), ('cdn.adsafeprotected.com', 1), ('newrevive.detik.com', 1), ('www.gsta


 197 douban.com [('img3.doubanio.com', 65), ('img1.doubanio.com', 3), ('www.douban.com', 1)]

 198 scribd.com [('s-f.scribdassets.com', 25), ('www.scribd.com', 2), ('', 1), ('cdn.ravenjs.com', 1), ('apis.google.com', 1), ('static.criteo.net', 1)]

 199 cnet.com [('cnet2.cbsistatic.com', 29), ('cnet4.cbsistatic.com', 28), ('cnet3.cbsistatic.com', 26), ('cnet1.cbsistatic.com', 24), ('static.chartbeat.com', 1), ('www.googletagservices.com', 1), ('js-sec.indexww.com', 1), ('mtrx.go.sonobi.com', 1), ('www.cnet.com', 1), ('', 1)]

 200 patria.org.ve [('cdn.patria.org.ve', 52), ('', 24), ('www.google.com', 1)]

 201 indiatimes.com [('im.indiatimes.in', 85), ('staticresources.indiatimes.in', 16), ('media.indiatimes.in', 6), ('static.clmbtech.com', 1), ('www.indiatimes.com', 1), ('', 1)]

 202 tvbs.com.tw [('cc.tvbs.com.tw', 52), ('s3-ap-northeast-1.amazonaws.com', 36), ('imagecache.tvbs.com.tw', 2), ('d5nxst8fruw4z.cloudfront.net', 1)]

 203 jianshu.com [('cdn2.jianshu.io', 23), ('upload.jian


 275 archive.org [('archive.org', 19), ('', 1)]

 276 forbes.com [('i.forbesimg.com', 10), ('cdn.speedcurve.com', 1), ('native.sharethrough.com', 1), ('thumbor.forbes.com', 1)]

 277 1337x.to False

 278 ebc.net.tw [('', 69), ('unpkg.com', 3), ('cdn.jsdelivr.net', 2), ('cdnjs.cloudflare.com', 1), ('d5nxst8fruw4z.cloudfront.net', 1)]

 279 liputan6.com [('cdn0-production-images-kly.akamaized.net', 309), ('cdn-production-assets-kly.akamaized.net', 255), ('cdn1-production-images-kly.akamaized.net', 199), ('ajax.googleapis.com', 1), ('cdn.onesignal.com', 1), ('www.google-analytics.com', 1), ('cdn0-production-assets-kly.akamaized.net', 1), ('cdns.klimg.com', 1), ('com.woi.liputan6.android', 1)]

 280 momoshop.com.tw []

 281 weather.com [('s.w-x.co', 24), ('', 9), ('cdn.polyfill.io', 1), ('weather.com', 1), ('www.googletagservices.com', 1)]

 282 resentaticexhaus.info False

 283 thesaurus.com [('images.ctfassets.net', 13), ('www.thesaurus.com', 6), ('ads.pubmatic.com', 1), ('www.googletag


 347 intuit.com [('', 13)]

 348 samsung.com [('images.samsung.com', 39), ('cdn.samsung.com', 10), ('assets.adobedtm.com', 1)]

 349 mercadolibre.com.ar [('http2.mlstatic.com', 22)]

 350 xfinity.com [('cdn.comcast.com', 12), ('cdn.pdc.comcast.com', 6), ('assets.adobedtm.com', 1), ('xapi.xfinity.com', 1)]

 351 aol.com [('s.aolcdn.com', 46), ('s.blogsmithmedia.com', 20), ('o.aolcdn.com', 15), ('s.yimg.com', 3), ('consent.cmp.oath.com', 1), ('delivery.vidible.tv', 1)]

 352 outbrain.com [('', 32)]

 353 kakaku.com [('img1.kakaku.k-img.com', 7), ('', 2), ('css1.kakaku.k-img.com', 2), ('ads.rubiconproject.com', 1), ('www.googletagservices.com', 1), ('assets.adobedtm.com', 1), ('script1.kakaku.k-img.com', 1)]

 354 youku.com [('static.youku.com', 252), ('image.uc.cn', 12), ('g.alicdn.com', 6), ('ykimg.alicdn.com', 5), ('img.alicdn.com', 2), ('js.ykimg.com', 1), ('', 1)]

 355 evernote.com [('', 37), ('cdn.optimizely.com', 1)]

 356 behance.net [('a5.behance.net', 1)]

 357 17ok.com [('img


 426 patreon.com [('c5.patreon.com', 7), ('c1.patreon.com', 4), ('cdnjs.cloudflare.com', 2), ('cdn.ravenjs.com', 1), ('www.google-analytics.com', 1)]

 427 shaparak.ir [('', 36)]

 428 upwork.com [('', 17), ('assets.static-upwork.com', 3)]

 429 leboncoin.fr []

 430 healthline.com [('d3otfkap419ijo.cloudfront.net', 6), ('i0.wp.com', 5), ('www.healthline.com', 2), ('prod.adspsp.com', 1), ('www.googletagservices.com', 1), ('www.googletagmanager.com', 1), ('c.amazon-adsystem.com', 1), ('c.lytics.io', 1)]

 431 baike.com [('static.hudong.com', 38), ('www.baike.com', 10), ('www.huimg.cn', 9), ('', 5), ('www.google-analytics.com', 1), ('www.googleadservices.com', 1), ('a1.att.hudong.com', 1)]

 432 chegg.com [('c.cheggcdn.com', 4), ('components.cheggcdn.com', 4), ('assets.cheggcdn.com', 2), ('assets.adobedtm.com', 1)]

 433 wp.pl [('i.wpimg.pl', 20)]

 434 skype.com [('secure.skypeassets.com', 17), ('statics-uhf-eus.akamaized.net', 1), ('www.microsoft.com', 1)]

 435 onet.pl [('ocdn.eu', 5


 495 oath.com [('s.blogsmithmedia.com', 34), ('s.aolcdn.com', 18), ('s.yimg.com', 4), ('munchkin.marketo.net', 1), ('www.googleadservices.com', 1)]

 496 jf71qh5v14.com False

 497 souq.com [('cf1.s3.souqcdn.com', 147), ('', 11), ('cms.souqcdn.com', 9), ('cf5.s3.souqcdn.com', 4), ('cf2.s3.souqcdn.com', 2), ('cf4.s3.souqcdn.com', 2), ('souqcms.s3.amazonaws.com', 1), ('production-upload-temp-images.s3.amazonaws.com', 1)]

 498 biobiochile.cl [('', 36), ('libs.biobiochile.cl', 17), ('www.googletagservices.com', 5), ('www.biobiochile.cl', 5), ('www.gstatic.com', 2), ('partner.googleadservices.com', 1), ('d5nxst8fruw4z.cloudfront.net', 1)]

 499 merdeka.com [('cdns.klimg.com', 83), ('cdn.onesignal.com', 1), ('code.jquery.com', 1), ('www.googleadservices.com', 1), ('www.newshub.id', 1), ('d5nxst8fruw4z.cloudfront.net', 1), ('cdn0-a.production.images.static6.com', 1), ('', 1)]

 500 adp.com [('', 56), ('cdn.optimizely.com', 1)]


In [147]:
import difflib
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

from CDNdomains import cdn_domains
THRESH = 0.8

def find_cdn_by_url(url):
    for cdn_url in cdn_domains.keys():
        r = similar(cdn_url, url)
        if r>THRESH:
            print("\t", r, url, cdn_url, end="|")
            return cdn_url
    return False

In [ ]:
cnt = get_homepage_url_count(site)
cdn_cnt = defaultdict(int)
cdn_flag = False
cdn_name = False

print(site)

if cnt is None:
    print("False, Nothing counted on page")
else:
    sum_url_cnt = sum([url_cnt for url, url_cnt in cnt])
    for url, url_cnt in cnt:
        if url == "":
            cdn_name = "local"
        else:
            cdn_name = find_cdn_by_url(url)
            if cdn_name:
                cdn_flag = True
            if not (cdn_name):
                if "cdn" in url:
                    parts = url.split('.')
                    for i in range(len(parts)):
                        if 'cdn' in parts[i]:
                            break
                    cdn_name = '.'.join(parts[i:])
                elif similar(site, url) > THRESH:
                    cdn_name = "local" 

In [59]:
def find_cdn_by_counting_url(site):
    """finds probable cdn based on most count of url static resources"""
    cnt = get_homepage_url_count(site)
    cdn_cnt = defaultdict(int)

    if cnt:
        sum_url_cnt = sum([url_cnt for url, url_cnt in cnt])
        # print(site+":", end="")
        # print(cnt[:4])
        # print(sum_url_cnt)

        for url, url_cnt in cnt:
            url_score = url_cnt/sum_url_cnt
            
            if url == '':  # local links
                cdn_name = ''
                

            else:
                cdn_name = find_cdn_by_url(url)

                if not (cdn_name):
                    # if '.cdn' in url:
                    #    #split at first occurrence of 'cdn' and print everything after
                    #    cdn_name = ''.join(url.partition('.cdn')[1:])  # unknown CDN to be resolved later
                    # elif 'cdn.' in url:
                    #    #split at last '.' before cdn (regex is better for this)
                    #    url_r = url[::-1]
                    #    cdn_name = ''.join(url.partition('.ndc')[1:])

                    if 'cdn' in url:
                        # domain itself contains the cdn word
                        parts = url.split('.')
                        for i in range(len(parts)):
                            if 'cdn' in parts[i]:
                                break
                        cdn_name = '.'.join(parts[i:])

                    #TODO very simple logic - change to use url_count
                    # say if url_cnt/sum_url_cnt > 1/5 then cdn_name = url since 1/5th of traffic is going there anyway
                    # eg: rgstatic.net and researchgate.net will be separated here while rgstatic.net will be ignored
                    elif site in url:
                        cdn_name = ''  # local site resources and links
                    else:
                        if url_cnt > sum_url_cnt/5.0:
                            cdn_name = url
                        else:
                            cdn_name = '?'  # unknown external links

            cdn_cnt[cdn_name] += url_cnt

        # print(cdn_cnt)
        # sort by value into list [('cdn1': n1), ('cdn2': n2), ('?': n3), ...]
        cdn_cnt_sorted = sorted(cdn_cnt.items(), key=lambda x: x[1], reverse=True)

        """save data"""
        temp_data['url_cnt'] = cdn_cnt_sorted
        """end save"""

        # remove "?" as option --> random external link with not enough (1/5th) traffic
        cdn_cnt_filtered = [i for i in cdn_cnt_sorted if i[0] != '?']

        if len(cdn_cnt_filtered) > 0:
            # for c,n in cdn_cnt_filtered:
            #    if (site in c):
            #        return c

            sum_cdn_links = sum([i[1] for i in cdn_cnt_filtered])

            i = 0
            probable_cdn = cdn_cnt_filtered[i][0]
            # enter loop only if first choice is '' else return first choice
            while (probable_cdn == ''):
                i += 1
                if i < len(cdn_cnt_filtered):
                    probable_cdn = cdn_cnt_filtered[i][0]

                    if (cdn_cnt_filtered[i][1] <= 2):
                        # if num url is too low
                        i = 0
                        probable_cdn = cdn_cnt_filtered[i][0]
                        break

                    if probable_cdn in major_cdns:
                        # if num famous url is high but can't compare with other links
                        # same logic 1/5 traffic but this time only compare with local links + potential CDNs no external
                        if (cdn_cnt_filtered[i][1] >= sum_cdn_links / 5.):
                            # 1/10 was arbitarily decided but surprisingly was the threshold for deciding whatsapp, stackoverflow
                            # even if it is local, break loop and return
                            break
                        else:
                            # if links to Google etc much less than sum then set back to first element regardless of ''
                            i = 0
                            probable_cdn = cdn_cnt_filtered[i][0]
                            break
                            # problem only if unknown cdn follows Google (ex: 9gag, whatsapp)

                else:
                    # if i>len set back to first element regardless of '' or [Google, fb, twitter] or whatever
                    i = 0
                    probable_cdn = cdn_cnt_filtered[i][0]
                    break

            print('\t' + site + ': ' + probable_cdn + ' | ' + str(sum_cdn_links) + ': ', end="")
            print(cdn_cnt_filtered)

            """save data"""
            temp_data['sum_cdn_links'] = sum_cdn_links
            temp_data['probable_cdn'] = probable_cdn

            # _save_cdn_counter_detection(temp_data)
            """end save"""

            return probable_cdn

    return False

In [179]:
import re

def split_before(pattern,text):
    prev = 0
    for m in re.finditer(pattern,text):
        yield text[prev:m.start()]
        prev = m.start()
    yield text[prev:]

In [192]:
url = "abc.cdn.twitter.com"
pattern = "cdn"

x = list( split_before(pattern, url) )
if x[0]=="":
    print(''.join(x))
else:
    print(x[0].split(".")[-1] + ''.join(x[1:]))
    pass

cdn.twitter.com


In [178]:
text = "ab123.twittercdn.com"
for m in re.finditer("cdn",text):
    print(m.start())

13


In [ ]:
sorted()